# 심층 합성곱 생성적 적대 신경망 (Deep Convolutional Generative Adversarial Networks, DCGAN)

In [ ]:
# GIF를 만들기위해 설치합니다.

### 데이터세트 로드 및 준비하기

생성자와 감별자를 훈련하기위해 MNIST 데이터셋을 사용할것입니다. 생성자는 손글씨 숫자 데이터를 닮은 숫자들을 생성할 것입니다.

In [ ]:
# mnist dataset load
#(train_images, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# 데이터 배치를 만들고 섞습니다.

## 모델 만들기

### 생성자

생성자는 시드값(임의의 노이즈)에서 이미지를 생성하기 위해, `tf.keras.layers.Conv2DTranspose`(업샘플링) 레이어를 이용합니다. 처음 `Dense` 레이어는 해당 시드값을 입력값으로 받습니다. 다음으로 원하는 크기인 28x28x1의 이미지에 도달할 때까지 업샘플링을 여러 번 수행합니다. tanh 함수를 사용하는 출력 레이어를 제외한 각 레이어에서 `tf.keras.layers.LeakyReLU` 활성화가 사용됩니다.

In [ ]:
def make_generator_model():
    # 입력 차원: 100, 출력 차원: 7*7*256
    # 1차원 출력을 3차원으로 변환 (7x7x256)
    # 역 컨볼루션 레이어 추가 (128 필터, 5x5 커널)
    # 역 컨볼루션 레이어 추가 (64 필터, 5x5 커널)
    # 최종 역 컨볼루션 레이어 (1 필터, 5x5 커널, 'tanh' 활성화)

(아직 훈련이 되지않은) 생성자를 이용해 이미지를 생성해봅시다.

### 감별자

감별자는 합성곱 신경망(Convolutional Neural Network, CNN) 기반의 이미지 분류기입니다.

In [ ]:
def make_discriminator_model():
    # 64 필터, 5x5 커널, 2x2 스트라이드, 입력 형태 [28, 28, 1]
    # 128 필터, 5x5 커널, 2x2 스트라이드

(아직까지 훈련이 되지 않은) 감별자를 사용하여, 생성된 이미지가 진짜인지 가짜인지 판별합니다. 모델은 진짜 이미지에는 양수의 값 (positive values)을, 가짜 이미지에는 0을 출력하도록 훈련되어집니다.

## 손실과 옵티마이저 정의하기

두 모델의 손실 함수와 옵티마이저를 정의합니다.


In [ ]:
# 이 메서드는 크로스 엔트로피 손실함수 (cross entropy loss)를 계산하기 위해 헬퍼 (helper) 함수를 반환합니다.

### 감별자 손실함수

이 메서드는 감별자가 가짜 이미지에서 얼마나 진짜 이미지를 잘 판별하는지 수치화합니다. 진짜 이미지에 대한 감별자의 예측과 1로 이루어진 행렬을 비교하고, 가짜 (생성된) 이미지에 대한 감별자의 예측과 0으로 이루어진 행렬을 비교합니다.

In [ ]:
def discriminator_loss(real_output, fake_output):
    # 진짜 이미지에 대한 손실 계산
    # 진짜 이미지 출력(real_output)이 1(진짜)로 인식되도록 하는 손실
    # 가짜 이미지에 대한 손실 계산
    # 가짜 이미지 출력(fake_output)이 0(가짜)으로 인식되도록 하는 손실
    # 총 손실은 진짜 이미지 손실과 가짜 이미지 손실의 합

### 생성자 손실함수

생성자의 손실은 판별자를 얼마나 잘 속일 수 있었는지를 수치화합니다. 직관적으로, 생성자가 잘 작동하면 판별자는 가짜 이미지를 실제(또는 1)로 분류합니다. 여기에서 생성된 이미지에 대한 판별자의 판단을 1의 배열과 비교합니다.

In [ ]:
def generator_loss(fake_output):
    # 가짜 이미지에 대한 손실 계산
    # 생성자가 생성한 가짜 이미지(fake_output)가 진짜(1)로 인식되도록 하는 손실

판별자와 생성자 옵티마이저는 두 네트워크를 별도로 훈련하기 때문에 서로 다릅니다.

### 체크포인트 저장

이 노트북은 오랫동안 진행되는 훈련이 중단된 경우 유용한 모델의 저장 및 복구 방법을 보여줍니다.

In [ ]:
# 체크포인트를 저장할 디렉토리 설정
# 생성자와 판별자의 옵티마이저 및 모델 상태를 저장합니다

## 훈련 루프 정의하기


In [ ]:
# 이 시드를 시간이 지나도 재활용합니다.
# (GIF 애니메이션에서 진전 내용을 시각화하는데 쉽기 때문)

생성자가 입력으로 임의의 시드를 받으면 훈련 루프가 시작됩니다. 해당 시드는 이미지를 생성하는 데 사용됩니다. 감별자를 사용하여 (훈련 세트에서 가져온) 진짜 이미지와 (생성자를 통해 생성된) 가짜 이미지를 분류합니다. 각 모델의 손실을 계산하고, 그래디언트를 사용해 생성자와 감별자를 업데이트합니다.

In [ ]:
# `tf.function`이 어떻게 사용되는지 주목해 주세요.
# 이 데코레이터는 함수를 "컴파일"합니다.
def train_step(images):
        # 생성자를 사용하여 이미지 생성
        # 판별자를 사용하여 진짜 이미지와 가짜 이미지 평가
        # 생성자와 판별자의 손실 계산
    # 생성자와 판별자의 그래디언트 계산
    # 계산된 그래디언트를 이용하여 옵티마이저 업데이트

In [ ]:
def train(dataset, epochs):
    # GIF를 위한 이미지를 바로 생성
    # 15 에포크가 지날 때마다 모델을 저장
    # print (' 에포크 {} 에서 걸린 시간은 {} 초 입니다'.format(epoch +1, time.time()-start))
  # 마지막 에포크가 끝난 후 생성

**이미지 생성 및 저장**


In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # training = False - (배치정규화 포함) 모든 층이 추론 모드로 실행

## 모델 훈련

위에서 정의한 `train()` 메서드를 호출하여 생성자와 판별자를 동시에 훈련합니다. GAN 훈련은 까다로울 수 있습니다. 생성자와 판별자가 서로를 압도하지 않는 것이 중요합니다(예: 비슷한 속도로 훈련됨).

훈련 초기에 생성된 이미지는 무작위 노이즈처럼 보입니다. 훈련이 진행됨에 따라 생성된 숫자는 점점 더 실제처럼 보일 것입니다. 약 50 epoch 후에는 MNIST 숫자와 유사합니다. Colab의 기본 설정에서 이 작업은 epoch당 약 1분 정도 소요될 수 있습니다.

마지막 체크포인트를 복구합니다.

## GIF 생성


In [ ]:
# 에포크 숫자를 사용하여 하나의 이미지를 보여줍니다.
def display_image(epoch_no):

`imageio`로 훈련 중에 저장된 이미지를 사용해 GIF 애니메이션을 만듭니다.

In [ ]:
# imageio 라이브러리를 사용하여 애니메이션 파일을 쓰는 작업 시작
    # 'image*.png' 패턴을 가진 모든 파일의 이름을 가져옵니다.
    # 파일 이름을 정렬합니다. (예: image1.png, image2.png, ...)
        # 각각의 이미지 파일을 읽어옵니다.
        # 읽은 이미지를 애니메이션에 추가
    # 마지막 이미지를 한 번 더 애니메이션에 추가. (원하는 특정 효과를 위해 같은 이미지를 2번 연속으로 넣음.)

tensorflow_docs.vis.embed 모듈의 embed_file 함수는 이미지 파일을 HTML에 임베드하는 함수입니다. 이 함수는 이미지 파일의 URL 또는 경로를 인자로 받아 HTML에 임베드할 태그를 반환합니다.  
Jupyter Notebook은 웹 기반의 대화식 컴퓨팅 환경이므로, 셀의 출력부에 나타나는 모든 것은 기본적으로 HTML로 렌더링될 수 있습니다.   
따라서, embed.embed_file(anim_file)를 Jupyter Notebook의 셀에서 실행하면, 해당 셀의 아래 출력 영역에 anim_file에 해당하는 애니메이션 또는 이미지가 나타나게 됩니다.